In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive/')
os.chdir('/content/drive/My Drive/BPM')
root = os.getcwd()
!pwd

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/.shortcut-targets-by-id/1rGkjnEE_3Wi6ptkR36DxTvGqAlmsVmg0/BPM


In [ ]:
import gzip
import json
import pandas as pd
from tqdm import tqdm_notebook as tqdm

# Download data

The two sub-categories are downloaded directly from the SNAP site and extracted with gzip

In [ ]:
os.chdir("./raw_data")
os.system("wget -c http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/meta_Electronics.json.gz")
os.system("gzip -d meta_Electronics.json.gz")

os.system("wget -c http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/meta_Office_Products.json.gz")
os.system("gzip -d meta_Office_Products.json.gz")

512

A unique csv file is generated from the two json file obtained

In [ ]:
#Creating an empty list
data = []

#Decoding the gzip file
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield json.dumps(eval(l))

#Defining f as the file that will contain json data
f = open("output_strict.json", 'w')

#Defining linebreak as '\n' and writing one at the end of each line
for l in parse(root + "/raw_data/meta_Electronics.json.gz"):
  f.write(l + '\n')

for l in parse(root + "/raw_data/meta_Office_Products.json.gz"):
  f.write(l + '\n')

#Appending each json element to the empty 'data' list
with open('output_strict.json', 'r') as f:
    for l in tqdm(f):
        data.append(json.loads(l))

#Reading 'data' as a pandas dataframe
full = pd.DataFrame(data)
full.to_csv(root+'/amazon/full.csv', index=False)
full.shape

<ipython-input-36-154de2c9041c>:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for l in tqdm(f):


0it [00:00, ?it/s]

(633034, 9)

In [ ]:
print(data[0])

{'asin': '0132793040', 'imUrl': 'http://ecx.images-amazon.com/images/I/31JIPhp%2BGIL.jpg', 'description': 'The Kelby Training DVD Mastering Blend Modes in Adobe Photoshop CS5 with Corey Barker is a useful tool for becoming familiar with the use of blend modes in Adobe Photoshop. For those who are serious about mastering all that Photoshop has to offer, mastering blend modes is just as important as mastering layers.In this DVD tutorial, seasoned expert Corey Barker explores the function of blend modes in a variety of scenarios such as image restoration, sharpening, adjustments, special effects and much more. Since every project scenario is different, Corey encourages you to experiment with these blend modes by giving you the skills and confidence you need.', 'categories': [['Electronics', 'Computers & Accessories', 'Cables & Accessories', 'Monitor Accessories']], 'title': 'Kelby Training DVD: Mastering Blend Modes in Adobe Photoshop CS5 By Corey Barker'}


In [ ]:
print(full.iloc[705])

asin                                                  9983901269
imUrl          http://ecx.images-amazon.com/images/I/31eSNcWV...
description                                                  NaN
categories     [[Electronics, Computers & Accessories, Cables...
title          Professional Kingston MicroSD 2GB (2 Gigabyte)...
price                                                        NaN
salesRank                 {'Cell Phones & Accessories': 2071844}
related                                                      NaN
brand                                                        NaN
Name: 705, dtype: object


#Delete row with NaN

All the rows containing a NaN value or an empty string is deleted

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv(root+'/amazon/full.csv')
# Drop unuseful fields
df = df.drop(['asin', 'price', 'salesRank', 'related'], axis=1)

# Drop rows with NaN value in selected columns
df = df.replace(r'^\s*$', np.nan, regex=True)
df = df.dropna(subset=['title', 'description', 'categories', 'imUrl', 'brand'])

df = df.reset_index(drop=True)

df.to_csv(root+"/amazon/field_filtered.csv", index=False)
df

,imUrl,description,categories,title,brand
0,http://ecx.images-amazon.com/images/I/41kQsHT8...,The Barnes &amp;amp; Noble BNADPN31 Power Kit ...,"[['Electronics', 'eBook Readers & Accessories'...",Barnes &amp; Noble NOOK Power Kit in Carbon BN...,Barnes &amp; Noble
1,http://ecx.images-amazon.com/images/I/01eX9rjx...,"Slim, smart, and lightweight, this ingenious c...","[['Electronics', 'eBook Readers & Accessories'...",Groovy Protective Stand Cover: Custom Designed...,Barnes &amp; Noble
2,http://ecx.images-amazon.com/images/I/41K7ymN5...,Power up your device with this Barnes &amp; No...,"[['Electronics', 'eBook Readers & Accessories'...",Barnes &amp; Noble OV/HB-ADP Universal Power Kit,Barnes &amp; Noble
3,http://ecx.images-amazon.com/images/I/31Vl-YJ5...,The Barnes &amp;amp; Noble B505360439 Wren Quo...,"[['Electronics', 'Computers & Accessories', 'P...",Barnes &amp; Noble B505360439 Wren Quote Cover,Barnes &amp; Noble
4,http://ecx.images-amazon.com/images/I/31zdIqAs...,The Barnes &amp;amp; Noble B211-4023 Johnson Q...,"[['Electronics', 'Computers & Accessories', 'P...",Barnes &amp; Noble B211-4023 Johnson Quote Cover,Barnes &amp; Noble
...,...,...,...,...,...
179997,http://ecx.images-amazon.com/images/I/51u6OjRY...,EcoPlus Remanufactured Drum/Drum Kit for Broth...,"[['Office Products', 'Office & School Supplies...",EcoPlus Remanufactured Drum/Drum Kit for Broth...,EcoPlus
179998,http://ecx.images-amazon.com/images/I/51nKwm3O...,The moment you've been waiting for. Introducin...,"[['Office Products', 'Office & School Supplies...","Lilly Pulitzer 2014-2015 Agenda - Peelin' Out,...",Lilly Pulitzer
179999,http://ecx.images-amazon.com/images/I/61Rjx8xG...,NEW DISNEY FROZEN ELSA & ANNA SCHOOL SUPPLY 1...,"[['Office Products', 'Office & School Supplies...",NEW DISNEY FROZEN ELSA &amp; ANNA SCHOOL SUPPL...,Frozen
180000,http://ecx.images-amazon.com/images/I/41SFeM42...,"Continuous Length Removable Paper Label 2-3/7""...","[['Office Products', 'Office & School Supplies...",Cont Removable Paper Label,Brother


In [ ]:
df = pd.read_csv(root+"/amazon/field_filtered.csv")
print(df.shape)

(180002, 5)


##Check correctness

In [ ]:
nan_values = df[df['imUrl'].isna()]
print (nan_values)

Empty DataFrame
Columns: [imUrl, description, categories, title, brand]
Index: []


In [ ]:
nan_values = df[df['description'].isna()]
print (nan_values)

Empty DataFrame
Columns: [imUrl, description, categories, title, brand]
Index: []


In [ ]:
nan_values = df[df['title'].isna()]
print (nan_values)

Empty DataFrame
Columns: [imUrl, description, categories, title, brand]
Index: []


In [ ]:
nan_values = df[df['brand'].isna()]
print (nan_values)

Empty DataFrame
Columns: [imUrl, description, categories, title, brand]
Index: []


In [ ]:
nan_values = df[df['description'] == '']
print (nan_values)

Empty DataFrame
Columns: [imUrl, description, categories, title, brand]
Index: []


# Unescape

All the bad HTML sequences are correctly substituted

In [ ]:
os.getcwd()

'/content/drive/.shortcut-targets-by-id/1rGkjnEE_3Wi6ptkR36DxTvGqAlmsVmg0/BPM'

In [ ]:
df = pd.read_csv(root+"/amazon/field_filtered.csv")
df

,imUrl,description,categories,title,brand
0,http://ecx.images-amazon.com/images/I/41kQsHT8...,The Barnes &amp;amp; Noble BNADPN31 Power Kit ...,"[['Electronics', 'eBook Readers & Accessories'...",Barnes &amp; Noble NOOK Power Kit in Carbon BN...,Barnes &amp; Noble
1,http://ecx.images-amazon.com/images/I/01eX9rjx...,"Slim, smart, and lightweight, this ingenious c...","[['Electronics', 'eBook Readers & Accessories'...",Groovy Protective Stand Cover: Custom Designed...,Barnes &amp; Noble
2,http://ecx.images-amazon.com/images/I/41K7ymN5...,Power up your device with this Barnes &amp; No...,"[['Electronics', 'eBook Readers & Accessories'...",Barnes &amp; Noble OV/HB-ADP Universal Power Kit,Barnes &amp; Noble
3,http://ecx.images-amazon.com/images/I/31Vl-YJ5...,The Barnes &amp;amp; Noble B505360439 Wren Quo...,"[['Electronics', 'Computers & Accessories', 'P...",Barnes &amp; Noble B505360439 Wren Quote Cover,Barnes &amp; Noble
4,http://ecx.images-amazon.com/images/I/31zdIqAs...,The Barnes &amp;amp; Noble B211-4023 Johnson Q...,"[['Electronics', 'Computers & Accessories', 'P...",Barnes &amp; Noble B211-4023 Johnson Quote Cover,Barnes &amp; Noble
...,...,...,...,...,...
179997,http://ecx.images-amazon.com/images/I/51u6OjRY...,EcoPlus Remanufactured Drum/Drum Kit for Broth...,"[['Office Products', 'Office & School Supplies...",EcoPlus Remanufactured Drum/Drum Kit for Broth...,EcoPlus
179998,http://ecx.images-amazon.com/images/I/51nKwm3O...,The moment you've been waiting for. Introducin...,"[['Office Products', 'Office & School Supplies...","Lilly Pulitzer 2014-2015 Agenda - Peelin' Out,...",Lilly Pulitzer
179999,http://ecx.images-amazon.com/images/I/61Rjx8xG...,NEW DISNEY FROZEN ELSA & ANNA SCHOOL SUPPLY 1...,"[['Office Products', 'Office & School Supplies...",NEW DISNEY FROZEN ELSA &amp; ANNA SCHOOL SUPPL...,Frozen
180000,http://ecx.images-amazon.com/images/I/41SFeM42...,"Continuous Length Removable Paper Label 2-3/7""...","[['Office Products', 'Office & School Supplies...",Cont Removable Paper Label,Brother


In [ ]:
# df = pd.read_csv("cleaned_data/preprocessed.csv")
# df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
# df.to_csv("cleaned_data/preprocessed.csv", index=False)
# df

In [ ]:
from bs4 import BeautifulSoup
from html import unescape
from tqdm import tqdm

tqdm.pandas()

In [ ]:
# Applica la funzione BeautifulSoup a ciascun elemento della colonna 'title'
df['description'] = df['description'].progress_apply(lambda x: BeautifulSoup(unescape(x), 'lxml').text)
df['categories'] = df['categories'].progress_apply(lambda x: BeautifulSoup(unescape(x), 'lxml').text)
df['title'] = df['title'].progress_apply(lambda x: BeautifulSoup(unescape(x), 'lxml').text)
df['brand'] = df['brand'].progress_apply(lambda x: BeautifulSoup(unescape(x), 'lxml').text)

df.to_csv("amazon/unescaped.csv", index=False)

# Stampa il DataFrame aggiornato
df

  0%|          | 0/180002 [00:00<?, ?it/s]<ipython-input-30-2e7d5e0c8a14>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  df['description'] = df['description'].progress_apply(lambda x: BeautifulSoup(unescape(x), 'lxml').text)
 19%|█▉        | 34316/180002 [00:10<00:35, 4092.66it/s]<ipython-input-30-2e7d5e0c8a14>:2: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  df['description'] = df['description'].progress_apply(lambda x: BeautifulSoup(unescape(x), 'lxml').text)
  0%|          | 0/180002 [00:00<?, ?it/s]<ipython-input-30-2e7d5e0c8a14>:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  df['categories'] = df['c

,imUrl,description,categories,title,brand
0,http://ecx.images-amazon.com/images/I/41kQsHT8...,The Barnes & Noble BNADPN31 Power Kit in Carbo...,"[['Electronics', 'eBook Readers & Accessories'...",Barnes & Noble NOOK Power Kit in Carbon BNADPN31,Barnes & Noble
1,http://ecx.images-amazon.com/images/I/01eX9rjx...,"Slim, smart, and lightweight, this ingenious c...","[['Electronics', 'eBook Readers & Accessories'...",Groovy Protective Stand Cover: Custom Designed...,Barnes & Noble
2,http://ecx.images-amazon.com/images/I/41K7ymN5...,Power up your device with this Barnes & Noble ...,"[['Electronics', 'eBook Readers & Accessories'...",Barnes & Noble OV/HB-ADP Universal Power Kit,Barnes & Noble
3,http://ecx.images-amazon.com/images/I/31Vl-YJ5...,The Barnes & Noble B505360439 Wren Quote Cover...,"[['Electronics', 'Computers & Accessories', 'P...",Barnes & Noble B505360439 Wren Quote Cover,Barnes & Noble
4,http://ecx.images-amazon.com/images/I/31zdIqAs...,The Barnes & Noble B211-4023 Johnson Quote Cov...,"[['Electronics', 'Computers & Accessories', 'P...",Barnes & Noble B211-4023 Johnson Quote Cover,Barnes & Noble
...,...,...,...,...,...
179997,http://ecx.images-amazon.com/images/I/51u6OjRY...,EcoPlus Remanufactured Drum/Drum Kit for Broth...,"[['Office Products', 'Office & School Supplies...",EcoPlus Remanufactured Drum/Drum Kit for Broth...,EcoPlus
179998,http://ecx.images-amazon.com/images/I/51nKwm3O...,The moment you've been waiting for. Introducin...,"[['Office Products', 'Office & School Supplies...","Lilly Pulitzer 2014-2015 Agenda - Peelin' Out,...",Lilly Pulitzer
179999,http://ecx.images-amazon.com/images/I/61Rjx8xG...,NEW DISNEY FROZEN ELSA & ANNA SCHOOL SUPPLY 1...,"[['Office Products', 'Office & School Supplies...",NEW DISNEY FROZEN ELSA & ANNA SCHOOL SUPPLY ST...,Frozen
180000,http://ecx.images-amazon.com/images/I/41SFeM42...,"Continuous Length Removable Paper Label 2-3/7""...","[['Office Products', 'Office & School Supplies...",Cont Removable Paper Label,Brother


#Check if url exists

A lot of URL sent in this dataset are not valid anymore. So the following code apply a check on the HTML response and delete the products containing an invalid URL

In [ ]:
!pip install datasets
!pip install "tqdm>=4.9.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 18.2 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
import pandas as pd
import requests
from tqdm import tqdm

tqdm.pandas()

In [ ]:
import pandas as pd
import numpy as np
import requests
from tqdm import tqdm
from urllib.parse import urlparse
import os

# Funzione per verificare se un URL è valido
def check_url_validity(url):
    try:
        response = requests.head(url)
        return response.status_code == 200
    except requests.exceptions.RequestException:
        return False

# Lettura del file CSV in un DataFrame
df = pd.read_csv(root+"/amazon/unescaped.csv")

# Generazione di 10,000 indici casuali
num_samples = 10000
sample_indices = np.random.choice(df.shape[0], num_samples, replace=False)

# Selezione del campione casuale dal DataFrame
sample_df = df.iloc[sample_indices]

# Estrazione degli URL dal campione
urls = sample_df['imUrl'].tolist()

# Filtraggio degli URL validi con tqdm
valid_urls = []
for url in tqdm(urls, desc="Checking URLs"):
    if check_url_validity(url):
        valid_urls.append(url)

print(len(valid_urls))

Checking URLs: 100%|██████████| 10000/10000 [22:31<00:00,  7.40it/s]

10000


In [ ]:
print(len(valid_urls))

10000


In [ ]:
# Creazione di un nuovo DataFrame con gli URL validi
valid_df = df[df['imUrl'].isin(valid_urls)]

# Salvataggio del DataFrame risultante in un file CSV
valid_df.to_csv(os.path.join(root,"amazon", "random_valid_url.csv"), index=False)

# Visualizzazione del DataFrame risultante
valid_df

,imUrl,description,categories,title,brand
6,http://ecx.images-amazon.com/images/I/31VZYUGE...,The Dessin Cover in Noir for Nook Tablet helps...,"[['Electronics', 'eBook Readers & Accessories'...",Barnes & Noble Dessin Leather Cover for Nook C...,Barnes & Noble
32,http://ecx.images-amazon.com/images/I/41Ng0Xir...,This beautiful 8G owl design flash drive is a ...,"[['Electronics', 'Computers & Accessories', 'D...",8GB Owl Design USB Flash Drive,Panda
43,http://ecx.images-amazon.com/images/I/41LdU7ps...,Charger Description: This high quality EconMax...,"[['Electronics', 'Camera & Photo', 'Accessorie...",NB-6L Charger For Canon PowerShot D10 SD3500IS...,EconMax
45,http://ecx.images-amazon.com/images/I/41kRFI9B...,"You are viewing a 80"" Portable Floor Stand Pro...","[['Electronics', 'Television & Video', 'Projec...",Floor Stand Pop Up Portable Projector Screen 8...,Generic
70,http://ecx.images-amazon.com/images/I/414CLZEH...,Professional Ultra SanDisk 16GB MicroSDHC Noki...,"[['Electronics', 'Computers & Accessories', 'C...",Professional Ultra SanDisk 16GB MicroSDHC Card...,SanDisk
...,...,...,...,...,...
179872,http://ecx.images-amazon.com/images/I/21yLIEPD...,"Premium hybrid mechanical pencil + stylus, com...","[['Office Products', 'Office & School Supplies...",Rotring 800+ Mechanical Pencil and Stylus Hybr...,Rotring
179894,http://ecx.images-amazon.com/images/I/41BHsXkS...,Genuine KODAK 30 Series Printhead. Buy your pr...,"[['Office Products', 'Office & School Supplies...",Kodak 7433402 30 Series Printhead,Kodak
179916,http://ecx.images-amazon.com/images/I/41eIcSw5...,"HQ Supplies is proud to offer its ""New Arrival...","[['Office Products', 'Office & School Supplies...",HQ Supplies © HP 124A HP Q6000A Q6001A Q6002A ...,HQ Supplies
179972,http://ecx.images-amazon.com/images/I/41NMFEue...,Description:The gooseneck mount with two holde...,"[['Office Products', 'Office & School Supplies...","Neewer® Gooseneck 7"" Inch-9.7"" Inch /17.7cm-24...",Neewer


# Create dataset for text2text model

Now the dataset to give as input to the text2text model has to be prepared. The format is the following: \<title, keywords -> text\>

N.B. the keywords are extracted from the  description

In [ ]:
df = pd.read_csv(root+"/amazon/random_valid_url.csv")

In [ ]:
small = df.iloc[:10000]
small

,imUrl,description,categories,title,brand
0,http://ecx.images-amazon.com/images/I/31VZYUGE...,The Dessin Cover in Noir for Nook Tablet helps...,"[['Electronics', 'eBook Readers & Accessories'...",Barnes & Noble Dessin Leather Cover for Nook C...,Barnes & Noble
1,http://ecx.images-amazon.com/images/I/41Ng0Xir...,This beautiful 8G owl design flash drive is a ...,"[['Electronics', 'Computers & Accessories', 'D...",8GB Owl Design USB Flash Drive,Panda
2,http://ecx.images-amazon.com/images/I/41LdU7ps...,Charger Description: This high quality EconMax...,"[['Electronics', 'Camera & Photo', 'Accessorie...",NB-6L Charger For Canon PowerShot D10 SD3500IS...,EconMax
3,http://ecx.images-amazon.com/images/I/41kRFI9B...,"You are viewing a 80"" Portable Floor Stand Pro...","[['Electronics', 'Television & Video', 'Projec...",Floor Stand Pop Up Portable Projector Screen 8...,Generic
4,http://ecx.images-amazon.com/images/I/414CLZEH...,Professional Ultra SanDisk 16GB MicroSDHC Noki...,"[['Electronics', 'Computers & Accessories', 'C...",Professional Ultra SanDisk 16GB MicroSDHC Card...,SanDisk
...,...,...,...,...,...
9995,http://ecx.images-amazon.com/images/I/318G%2Bg...,Note: You will receive a 100% comaptible scree...,"[['Electronics', 'Computers & Accessories', 'M...","SONY VAIO PCG-71318L LAPTOP LCD SCREEN 15.6"" W...",Sony
9996,http://ecx.images-amazon.com/images/I/51Zjq6FJ...,TARGUS TG-SC5490 Compact Camera Case with Stra...,"[['Electronics', 'Camera & Photo', 'Bags & Cas...",Targus TG-SC5490 Compact Camera Case with Stra...,Targus
9997,http://ecx.images-amazon.com/images/I/318G%2Bg...,Note: You will receive a 100% comaptible scree...,"[['Electronics', 'Computers & Accessories', 'L...","FUJITSU LIFEBOOK AH531 LAPTOP LCD SCREEN 15.6""...",Fujitsu
9998,http://ecx.images-amazon.com/images/I/31YYX1NV...,"10 Total outlets (5 surge protected, 5 battery...","[['Electronics', 'Computers & Accessories', 'C...",Apc Back-Ups Es 750va 10out 120 Volt Master Co...,APC


## Keywords extraction from description

In [ ]:
import spacy
import pandas as pd

nlp = spacy.load("en_core_web_sm")

dataset = pd.DataFrame()
dataset['title'] = small['title']
dataset['text'] = small['description']

tmp = []

for index, row in small.iterrows():
    keywords = [str(ent) for ent in nlp(row['description']).ents]
    keywords.append(row['brand'])
    tmp.append(keywords)

dataset['keywords'] = tmp

dataset.to_csv(root+"/amazon/dataset.csv", index=False)
dataset.head()

,title,text,keywords
0,Barnes & Noble Dessin Leather Cover for Nook C...,The Dessin Cover in Noir for Nook Tablet helps...,"[Noir, Nook Color, the Nook Tablet, Noir Cover..."
1,8GB Owl Design USB Flash Drive,This beautiful 8G owl design flash drive is a ...,"[8, Panda]"
2,NB-6L Charger For Canon PowerShot D10 SD3500IS...,Charger Description: This high quality EconMax...,"[EconMax NB-6L / NB6L Battery Charger, only 1...."
3,Floor Stand Pop Up Portable Projector Screen 8...,"You are viewing a 80"" Portable Floor Stand Pro...","[80, Portable Floor Stand Projector Screen, 80..."
4,Professional Ultra SanDisk 16GB MicroSDHC Card...,Professional Ultra SanDisk 16GB MicroSDHC Noki...,"[16, GB, 520, UHS-1 Class 10 Certified, sec, S..."


In [ ]:
result = pd.read_csv(root+"/amazon/dataset.csv")
result

,title,text,keywords
0,Barnes & Noble Dessin Leather Cover for Nook C...,The Dessin Cover in Noir for Nook Tablet helps...,"['Noir', 'Nook Color', 'the Nook Tablet', 'Noi..."
1,8GB Owl Design USB Flash Drive,This beautiful 8G owl design flash drive is a ...,"['8', 'Panda']"
2,NB-6L Charger For Canon PowerShot D10 SD3500IS...,Charger Description: This high quality EconMax...,"['EconMax NB-6L / NB6L Battery Charger', 'only..."
3,Floor Stand Pop Up Portable Projector Screen 8...,"You are viewing a 80"" Portable Floor Stand Pro...","['80', 'Portable Floor Stand Projector Screen'..."
4,Professional Ultra SanDisk 16GB MicroSDHC Card...,Professional Ultra SanDisk 16GB MicroSDHC Noki...,"['16', 'GB', '520', 'UHS-1 Class 10 Certified'..."
...,...,...,...
9995,"SONY VAIO PCG-71318L LAPTOP LCD SCREEN 15.6"" W...",Note: You will receive a 100% comaptible scree...,"['100%', 'Sony']"
9996,Targus TG-SC5490 Compact Camera Case with Stra...,TARGUS TG-SC5490 Compact Camera Case with Stra...,"['TARGUS TG-SC5490', 'Targus']"
9997,"FUJITSU LIFEBOOK AH531 LAPTOP LCD SCREEN 15.6""...",Note: You will receive a 100% comaptible scree...,"['100%', 'Fujitsu']"
9998,Apc Back-Ups Es 750va 10out 120 Volt Master Co...,"10 Total outlets (5 surge protected, 5 battery...","['10', '5', '5', 'APC']"


#Final

In [ ]:
import spacy
import pandas as pd
nlp = spacy.load("en_core_web_sm")

df = pd.read_csv(root+"/amazon/filtered_description_prompt.csv")
data = pd.DataFrame()
small = df.iloc[:10000]
data['keywords'] = [nlp(text).ents for text in small['prompt']]
print(data.head())
data['text'] = df['completion']
data['title'] = df['prompt']

                                            keywords
0  ((Computers, &, Accessories), (Cables, &, Acce...
1  ((Handhelds, &, Accessories), (Digital, Organi...
2  ((Audio, &, Video, Accessories), (Remote, Cont...
3  ((GPS, &, Navigation), (GPS), (Rand, McNally),...
4  ((Audio, &, Video, Accessories), (Headphones),...


In [ ]:
dt = pd.read_csv(root+"/amazon/keywords_description.csv")
dt = dt.loc[:, ~dt.columns.str.contains('^Unnamed')]
dt.head()

,keywords,text,title
0,"(Computers & Accessories, Cables & Accessories...",The Kelby Training DVD Mastering Blend Modes i...,Category is Electronics and Computers & Access...
1,"(Handhelds & Accessories, Digital Organizer, M...",Digital Organizer and Messenger,Category is Electronics and Computers & Access...
2,"(Audio & Video Accessories, Remote Controls, T...",The CLIKR-5 UR5U-8780L remote control is desig...,Category is Electronics and Accessories & Supp...
3,"(GPS & Navigation, GPS, Rand McNally, 52888146...","Like its award-winning predecessor, the Intell...",Category is Electronics and GPS & Navigation a...
4,"(Audio & Video Accessories, Headphones, Polaro...",Use these high quality headphones for internet...,Category is Electronics and Accessories & Supp...
